In [1]:
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras.optimizers import Adam


In [2]:

# Fake dataset
X = np.random.rand(1000, 20)
y = (np.sum(X[:, :10], axis=1) + 0.5 * np.sum(X[:, 10:], axis=1) > 10).astype(int)




In [3]:
# Base model 1: handles first 10 features
input1 = Input(shape=(10,))
x1 = Dense(32, activation='relu')(input1)
x1 = Dense(16, activation='relu')(x1)
out1 = Dense(8, activation='relu')(x1)
base_model1 = Model(inputs=input1, outputs=out1)



In [4]:
# Base model 2: handles last 10 features
input2 = Input(shape=(10,))
x2 = Dense(32, activation='relu')(input2)
x2 = Dense(16, activation='relu')(x2)
out2 = Dense(8, activation='relu')(x2)
base_model2 = Model(inputs=input2, outputs=out2)



In [5]:
# Meta-model: combines outputs from both base models
combined = Concatenate()([base_model1.output, base_model2.output])
z = Dense(16, activation='relu')(combined)
z = Dense(1, activation='sigmoid')(z)
meta_model = Model(inputs=[base_model1.input, base_model2.input], outputs=z)



In [6]:
meta_model.compile(optimizer=Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
# Split X into two parts for the base models
meta_model.fit([X[:, :10], X[:, 10:]], y, epochs=5, batch_size=32, verbose=1)



Epoch 1/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 656us/step - accuracy: 0.6090 - loss: 0.6865 
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 667us/step - accuracy: 0.9940 - loss: 0.3668
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 690us/step - accuracy: 0.9940 - loss: 0.0863
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 597us/step - accuracy: 0.9940 - loss: 0.0478
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 638us/step - accuracy: 0.9940 - loss: 0.0471


In [8]:
print("\nMeta Neural Network Evaluation:",
      meta_model.evaluate([X[:, :10], X[:, 10:]], y))

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 552us/step - accuracy: 0.9940 - loss: 0.0467

Meta Neural Network Evaluation: [0.04665948078036308, 0.9940000176429749]
